# Day 13 - claude

In [1]:
from typing import Tuple, List, Optional
import re

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]]:
    """
    Parse the input file and return a list of claw machine configurations.
    Each configuration contains ((A_x, A_y), (B_x, B_y), (prize_x, prize_y))
    """
    machines = []
    current_machine = []
    
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
        
    for i in range(0, len(lines), 3):
        # Parse button A
        a_match = re.match(r'Button A: X\+(\d+), Y\+(\d+)', lines[i])
        ax, ay = map(int, a_match.groups())
        
        # Parse button B
        b_match = re.match(r'Button B: X\+(\d+), Y\+(\d+)', lines[i + 1])
        bx, by = map(int, b_match.groups())
        
        # Parse prize location
        p_match = re.match(r'Prize: X=(\d+), Y=(\d+)', lines[i + 2])
        px, py = map(int, p_match.groups())
        
        machines.append(((ax, ay), (bx, by), (px, py)))
    
    return machines

def solve_machine(machine: Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]], 
                 max_presses: int = 100) -> Optional[Tuple[int, int]]:
    """
    Solve for a single machine. Returns (a_presses, b_presses) if solution exists,
    None otherwise. Ensures solutions are non-negative and within max_presses.
    """
    (ax, ay), (bx, by), (px, py) = machine
    
    # Try all possible combinations of button presses within the limits
    min_cost = float('inf')
    best_solution = None
    
    for a in range(max_presses + 1):
        for b in range(max_presses + 1):
            # Check if this combination solves both X and Y coordinates
            if (a * ax + b * bx == px) and (a * ay + b * by == py):
                # Calculate cost (A costs 3 tokens, B costs 1 token)
                cost = 3 * a + b
                if cost < min_cost:
                    min_cost = cost
                    best_solution = (a, b)
    
    return best_solution

def main():
    # Parse input
    machines = parse_input('input.txt')
    
    # Find solutions for each machine
    total_tokens = 0
    solvable_count = 0
    
    for i, machine in enumerate(machines, 1):
        solution = solve_machine(machine)
        if solution:
            a_presses, b_presses = solution
            tokens = 3 * a_presses + b_presses
            total_tokens += tokens
            solvable_count += 1
            print(f"Machine {i}: Solvable with {a_presses} A presses and {b_presses} B presses ({tokens} tokens)")
        else:
            print(f"Machine {i}: No solution exists")
    
    print(f"\nTotal prizes possible: {solvable_count}")
    print(f"Total tokens needed: {total_tokens}")

if __name__ == "__main__":
    main()

Machine 1: Solvable with 40 A presses and 93 B presses (213 tokens)
Machine 2: Solvable with 39 A presses and 3 B presses (120 tokens)
Machine 3: No solution exists
Machine 4: No solution exists
Machine 5: No solution exists
Machine 6: Solvable with 37 A presses and 31 B presses (142 tokens)
Machine 7: Solvable with 16 A presses and 5 B presses (53 tokens)
Machine 8: No solution exists
Machine 9: Solvable with 38 A presses and 40 B presses (154 tokens)
Machine 10: No solution exists
Machine 11: No solution exists
Machine 12: Solvable with 10 A presses and 60 B presses (90 tokens)
Machine 13: No solution exists
Machine 14: Solvable with 21 A presses and 68 B presses (131 tokens)
Machine 15: No solution exists
Machine 16: No solution exists
Machine 17: Solvable with 90 A presses and 76 B presses (346 tokens)
Machine 18: No solution exists
Machine 19: Solvable with 53 A presses and 15 B presses (174 tokens)
Machine 20: Solvable with 43 A presses and 77 B presses (206 tokens)
Machine 21: S

## Part 2

In [2]:
from typing import Tuple, List, Optional
import re
from math import gcd
from functools import reduce

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]]:
    """
    Parse the input file and return a list of claw machine configurations.
    Each configuration contains ((A_x, A_y), (B_x, B_y), (prize_x, prize_y))
    """
    machines = []
    current_machine = []
    
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
        
    for i in range(0, len(lines), 3):
        # Parse button A
        a_match = re.match(r'Button A: X\+(\d+), Y\+(\d+)', lines[i])
        ax, ay = map(int, a_match.groups())
        
        # Parse button B
        b_match = re.match(r'Button B: X\+(\d+), Y\+(\d+)', lines[i + 1])
        bx, by = map(int, b_match.groups())
        
        # Parse prize location
        p_match = re.match(r'Prize: X=(\d+), Y=(\d+)', lines[i + 2])
        px, py = map(int, p_match.groups())
        
        # Add the offset for Part 2
        px += 10000000000000
        py += 10000000000000
        
        machines.append(((ax, ay), (bx, by), (px, py)))
    
    return machines

def extended_gcd(a: int, b: int) -> Tuple[int, int, int]:
    """
    Returns (gcd, x, y) such that a * x + b * y = gcd
    """
    if a == 0:
        return b, 0, 1
    
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    
    return gcd, x, y

def solve_linear_diophantine(a: int, b: int, c: int) -> Optional[Tuple[int, int]]:
    """
    Solves the equation ax + by = c for integers x, y
    Returns None if no solution exists
    """
    # Find GCD and coefficients
    g, x0, y0 = extended_gcd(abs(a), abs(b))
    
    # Check if solution exists
    if c % g != 0:
        return None
    
    # Adjust signs
    if a < 0:
        x0 = -x0
    if b < 0:
        y0 = -y0
    
    # Scale to match c
    x0 *= c // g
    y0 *= c // g
    
    return (x0, y0)

def find_non_negative_solution(a: int, b: int, c: int) -> Optional[Tuple[int, int]]:
    """
    Find non-negative integer solution to ax + by = c
    Returns None if no non-negative solution exists
    """
    # Get base solution
    base = solve_linear_diophantine(a, b, c)
    if base is None:
        return None
    
    x0, y0 = base
    
    # Find parametric form coefficients
    g = gcd(a, b)
    t_step = abs(b) // g
    s_step = abs(a) // g
    
    # Find t that makes both x and y non-negative
    # x = x0 + t * (b/g)
    # y = y0 - t * (a/g)
    
    # Solve x ≥ 0 and y ≥ 0
    t_x = -(x0 // t_step) if x0 < 0 else 0
    t_y = (y0 // s_step) + 1 if y0 < 0 else 0
    
    t = max(t_x, t_y)
    
    x = x0 + t * t_step
    y = y0 - t * s_step
    
    if x < 0 or y < 0:
        return None
        
    return (x, y)

def solve_machine(machine: Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]) -> Optional[Tuple[int, int]]:
    """
    Solve for a single machine. Returns (a_presses, b_presses) if solution exists,
    None otherwise.
    """
    (ax, ay), (bx, by), (px, py) = machine
    
    # Solve for X coordinates
    x_solution = find_non_negative_solution(ax, bx, px)
    if not x_solution:
        return None
    
    # Solve for Y coordinates
    y_solution = find_non_negative_solution(ay, by, py)
    if not y_solution:
        return None
    
    # Check if solutions are compatible
    if x_solution[0] != y_solution[0] or x_solution[1] != y_solution[1]:
        return None
    
    return x_solution

def main():
    # Parse input
    machines = parse_input('input.txt')
    
    # Find solutions for each machine
    total_tokens = 0
    solvable_count = 0
    
    for i, machine in enumerate(machines, 1):
        solution = solve_machine(machine)
        if solution:
            a_presses, b_presses = solution
            tokens = 3 * a_presses + b_presses
            total_tokens += tokens
            solvable_count += 1
            print(f"Machine {i}: Solvable with {a_presses} A presses and {b_presses} B presses ({tokens} tokens)")
        else:
            print(f"Machine {i}: No solution exists")
    
    print(f"\nTotal prizes possible: {solvable_count}")
    print(f"Total tokens needed: {total_tokens}")

if __name__ == "__main__":
    main()

Machine 1: No solution exists
Machine 2: No solution exists
Machine 3: No solution exists
Machine 4: No solution exists
Machine 5: No solution exists
Machine 6: No solution exists
Machine 7: No solution exists
Machine 8: No solution exists
Machine 9: No solution exists
Machine 10: No solution exists
Machine 11: No solution exists
Machine 12: No solution exists
Machine 13: No solution exists
Machine 14: No solution exists
Machine 15: No solution exists
Machine 16: No solution exists
Machine 17: No solution exists
Machine 18: No solution exists
Machine 19: No solution exists
Machine 20: No solution exists
Machine 21: No solution exists
Machine 22: No solution exists
Machine 23: No solution exists
Machine 24: No solution exists
Machine 25: No solution exists
Machine 26: No solution exists
Machine 27: No solution exists
Machine 28: No solution exists
Machine 29: No solution exists
Machine 30: No solution exists
Machine 31: No solution exists
Machine 32: No solution exists
Machine 33: No so

In [3]:
from typing import Tuple, List, Optional
import re
from math import gcd

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]]:
    """Parse the input file and return a list of claw machine configurations."""
    machines = []
    
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
        
    for i in range(0, len(lines), 3):
        # Parse button A
        a_match = re.match(r'Button A: X\+(\d+), Y\+(\d+)', lines[i])
        ax, ay = map(int, a_match.groups())
        
        # Parse button B
        b_match = re.match(r'Button B: X\+(\d+), Y\+(\d+)', lines[i + 1])
        bx, by = map(int, b_match.groups())
        
        # Parse prize location
        p_match = re.match(r'Prize: X=(\d+), Y=(\d+)', lines[i + 2])
        px, py = map(int, p_match.groups())
        
        # Add the offset for Part 2
        px += 10000000000000
        py += 10000000000000
        
        machines.append(((ax, ay), (bx, by), (px, py)))
    
    return machines

def extended_gcd(a: int, b: int) -> Tuple[int, int, int]:
    """Extended Euclidean Algorithm."""
    if a == 0:
        return b, 0, 1
    
    d, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    
    return d, x, y

def solve_machine(machine: Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]) -> Optional[Tuple[int, int]]:
    """
    Solve for a single machine using system of equations.
    Returns (a_presses, b_presses) if solution exists, None otherwise.
    """
    (ax, ay), (bx, by), (px, py) = machine
    
    # Solve the system of equations:
    # ax * A + bx * B = px
    # ay * A + by * B = py
    
    # First equation coefficients
    d1, x1, y1 = extended_gcd(ax, bx)
    if px % d1 != 0:
        return None
    
    # Second equation coefficients
    d2, x2, y2 = extended_gcd(ay, by)
    if py % d2 != 0:
        return None
    
    # Find particular solutions
    k1 = px // d1
    k2 = py // d2
    
    # Initial solutions
    a1 = k1 * x1
    b1 = k1 * y1
    a2 = k2 * x2
    b2 = k2 * y2
    
    # Get the step sizes
    t1 = bx // d1
    t2 = by // d2
    
    # Try to find matching solutions
    # We need to find k where a1 + k*t1 = a2 + m*t2 for some m
    # and both solutions are non-negative
    
    # Calculate the range for k
    k_start = max(0, -(min(a1, a2) // max(t1, t2)) - 1)
    k_end = k_start + 10000000  # Reasonable upper limit
    
    for k in range(k_start, k_end):
        a = a1 + k * t1
        if a < 0:
            continue
            
        b = b1 - k * (ax // d1)
        if b < 0:
            continue
            
        # Verify both equations
        if (ax * a + bx * b == px) and (ay * a + by * b == py):
            return (a, b)
    
    return None

def main():
    machines = parse_input('input.txt')
    total_tokens = 0
    solvable_count = 0
    
    for i, machine in enumerate(machines, 1):
        solution = solve_machine(machine)
        if solution:
            a_presses, b_presses = solution
            tokens = 3 * a_presses + b_presses
            total_tokens += tokens
            solvable_count += 1
            print(f"Machine {i}: Solvable with {a_presses} A presses and {b_presses} B presses ({tokens} tokens)")
        else:
            print(f"Machine {i}: No solution exists")
    
    print(f"\nTotal prizes possible: {solvable_count}")
    print(f"Total tokens needed: {total_tokens}")

if __name__ == "__main__":
    main()

Machine 1: No solution exists
Machine 2: No solution exists
Machine 3: No solution exists
Machine 4: No solution exists
Machine 5: No solution exists
Machine 6: No solution exists
Machine 7: No solution exists
Machine 8: No solution exists
Machine 9: No solution exists
Machine 10: No solution exists
Machine 11: No solution exists
Machine 12: No solution exists


KeyboardInterrupt: 

In [6]:
from typing import Tuple, List, Optional
import re
from math import gcd

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]]:
    """Parse the input file and return a list of claw machine configurations."""
    machines = []
    
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
        
    for i in range(0, len(lines), 3):
        # Parse button A
        a_match = re.match(r'Button A: X\+(\d+), Y\+(\d+)', lines[i])
        ax, ay = map(int, a_match.groups())
        
        # Parse button B
        b_match = re.match(r'Button B: X\+(\d+), Y\+(\d+)', lines[i + 1])
        bx, by = map(int, b_match.groups())
        
        # Parse prize location
        p_match = re.match(r'Prize: X=(\d+), Y=(\d+)', lines[i + 2])
        px, py = map(int, p_match.groups())
        
        # Add the offset for Part 2
        px += 10000000000000
        py += 10000000000000
        
        machines.append(((ax, ay), (bx, by), (px, py)))
    
    return machines

def extended_gcd(a: int, b: int) -> Tuple[int, int, int]:
    """Extended Euclidean Algorithm."""
    if a == 0:
        return b, 0, 1
    
    d, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    
    return d, x, y

def solve_diophantine(a: int, b: int, c: int) -> Optional[Tuple[int, int, int, int]]:
    """
    Solves the Diophantine equation ax + by = c.
    Returns (x0, y0, dx, dy) where:
    - x0, y0 is a particular solution
    - dx, dy are the steps for generating more solutions
    Returns None if no solution exists.
    """
    d, x0, y0 = extended_gcd(a, b)
    if c % d != 0:
        return None
        
    x0 = x0 * (c // d)
    y0 = y0 * (c // d)
    dx = b // d
    dy = -a // d
    
    return (x0, y0, dx, dy)

def find_minimal_non_negative(x0: int, y0: int, dx: int, dy: int) -> Optional[Tuple[int, int]]:
    """Find the minimal non-negative solution to the Diophantine equation."""
    if dx == 0 and dy == 0:
        if x0 >= 0 and y0 >= 0:
            return (x0, y0)
        return None
        
    # Find k that makes both x and y non-negative and minimal
    k_x = -(x0 // dx) if dx != 0 and x0 < 0 else 0
    k_y = -(y0 // dy) if dy != 0 and y0 < 0 else 0
    k = max(k_x, k_y)
    
    x = x0 + k * dx
    y = y0 + k * dy
    
    if x >= 0 and y >= 0:
        return (x, y)
    return None

def solve_machine(machine: Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]) -> Optional[Tuple[int, int]]:
    """Solve for a single machine."""
    (ax, ay), (bx, by), (px, py) = machine
    
    # Solve x-coordinate equation: ax * A + bx * B = px
    x_sol = solve_diophantine(ax, bx, px)
    if not x_sol:
        return None
    
    # Solve y-coordinate equation: ay * A + by * B = py
    y_sol = solve_diophantine(ay, by, py)
    if not y_sol:
        return None
    
    x0, xb0, xda, xdb = x_sol
    y0, yb0, yda, ydb = y_sol
    
    # Find k and m such that:
    # x0 + k*xda = y0 + m*yda (A presses must match)
    # xb0 + k*xdb = yb0 + m*ydb (B presses must match)
    
    if xda == 0 or yda == 0:
        if x0 == y0 and xb0 == yb0:  # Check if particular solutions match
            solution = find_minimal_non_negative(x0, xb0, 0, 0)
            return solution
        return None
    
    # Solve for k using the first equation
    k = (y0 - x0) // xda if (y0 - x0) % xda == 0 else None
    if k is None:
        return None
    
    # Verify B presses match
    if xb0 + k * xdb != yb0:
        return None
    
    # Find minimal non-negative solution
    solution = find_minimal_non_negative(x0 + k * xda, xb0 + k * xdb, 0, 0)
    return solution

def main():
    machines = parse_input('input.txt')
    total_tokens = 0
    solvable_count = 0
    
    for i, machine in enumerate(machines, 1):
        solution = solve_machine(machine)
        if solution:
            a_presses, b_presses = solution
            tokens = 3 * a_presses + b_presses
            total_tokens += tokens
            solvable_count += 1
            print(f"Machine {i}: Solvable with {a_presses} A presses and {b_presses} B presses ({tokens} tokens)")
        else:
            print(f"Machine {i}: No solution exists")
    
    print(f"\nTotal prizes possible: {solvable_count}")
    print(f"Total tokens needed: {total_tokens}")

if __name__ == "__main__":
    main()

Machine 1: No solution exists
Machine 2: No solution exists
Machine 3: No solution exists
Machine 4: No solution exists
Machine 5: No solution exists
Machine 6: No solution exists
Machine 7: No solution exists
Machine 8: No solution exists
Machine 9: No solution exists
Machine 10: No solution exists
Machine 11: No solution exists
Machine 12: No solution exists
Machine 13: No solution exists
Machine 14: No solution exists
Machine 15: No solution exists
Machine 16: No solution exists
Machine 17: No solution exists
Machine 18: No solution exists
Machine 19: No solution exists
Machine 20: No solution exists
Machine 21: No solution exists
Machine 22: No solution exists
Machine 23: No solution exists
Machine 24: No solution exists
Machine 25: No solution exists
Machine 26: No solution exists
Machine 27: No solution exists
Machine 28: No solution exists
Machine 29: No solution exists
Machine 30: No solution exists
Machine 31: No solution exists
Machine 32: No solution exists
Machine 33: No so

In [ ]:
from typing import Tuple, List, Optional
import re
from math import gcd
from decimal import Decimal, getcontext

# Set precision high enough for our large numbers
getcontext().prec = 50

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]]:
    """Parse the input file and return a list of claw machine configurations."""
    machines = []
    
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
        
    for i in range(0, len(lines), 3):
        # Parse button A
        a_match = re.match(r'Button A: X\+(\d+), Y\+(\d+)', lines[i])
        ax, ay = map(int, a_match.groups())
        
        # Parse button B
        b_match = re.match(r'Button B: X\+(\d+), Y\+(\d+)', lines[i + 1])
        bx, by = map(int, b_match.groups())
        
        # Parse prize location
        p_match = re.match(r'Prize: X=(\d+), Y=(\d+)', lines[i + 2])
        px, py = map(int, p_match.groups())
        
        # Add the offset for Part 2
        px += 10000000000000
        py += 10000000000000
        
        machines.append(((ax, ay), (bx, by), (px, py)))
    
    return machines

def solve_machine(machine: Tuple[Tuple[int, int], Tuple[int, int], Tuple[int, int]]) -> Optional[Tuple[int, int]]:
    """
    Solve for a single machine using Gaussian elimination with fractions.
    Returns (a_presses, b_presses) if solution exists, None otherwise.
    """
    (ax, ay), (bx, by), (px, py) = machine
    
    # Convert to Decimal for precise arithmetic
    d_ax = Decimal(ax)
    d_ay = Decimal(ay)
    d_bx = Decimal(bx)
    d_by = Decimal(by)
    d_px = Decimal(px)
    d_py = Decimal(py)
    
    # Use determinant to solve the system
    det = d_ax * d_by - d_bx * d_ay
    if det == 0:
        return None
        
    # Solve for A and B
    a = (d_px * d_by - d_bx * d_py) / det
    b = (d_ax * d_py - d_px * d_ay) / det
    
    # Check if we got integer solutions
    if not (a % 1 == 0 and b % 1 == 0):
        return None
        
    # Convert to integers
    a_int = int(a)
    b_int = int(b)
    
    # Check for non-negative solutions
    if a_int < 0 or b_int < 0:
        return None
        
    # Verify solution
    if (ax * a_int + bx * b_int != px) or (ay * a_int + by * b_int != py):
        return None
        
    return (a_int, b_int)

def main():
    machines = parse_input('input.txt')
    total_tokens = 0
    solvable_count = 0
    
    for i, machine in enumerate(machines, 1):
        solution = solve_machine(machine)
        if solution:
            a_presses, b_presses = solution
            tokens = 3 * a_presses + b_presses
            total_tokens += tokens
            solvable_count += 1
            print(f"Machine {i}: Solvable with {a_presses} A presses and {b_presses} B presses ({tokens} tokens)")
        else:
            print(f"Machine {i}: No solution exists")
    
    print(f"\nTotal prizes possible: {solvable_count}")
    print(f"Total tokens needed: {total_tokens}")

if __name__ == "__main__":
    main()

Machine 1: No solution exists
Machine 2: No solution exists
Machine 3: Solvable with 210016155157 A presses and 113085622118 B presses (743134087589 tokens)
Machine 4: Solvable with 130293159841 A presses and 103642285990 B presses (494521765513 tokens)
Machine 5: Solvable with 152749491081 A presses and 105227427100 B presses (563475900343 tokens)
Machine 6: No solution exists
Machine 7: No solution exists
Machine 8: Solvable with 120452903158 A presses and 127680077124 B presses (489038786598 tokens)
Machine 9: No solution exists
Machine 10: Solvable with 133874239528 A presses and 105476673613 B presses (507099392197 tokens)
Machine 11: Solvable with 154494381994 A presses and 112359550835 B presses (575842696817 tokens)
Machine 12: No solution exists
Machine 13: Solvable with 143884892094 A presses and 104916067417 B presses (536570743699 tokens)
Machine 14: No solution exists
Machine 15: Solvable with 140643623515 A presses and 100119189517 B presses (522050060062 tokens)
Machine 

: 